# 作业要求

使用官方提供的示例，成功微调出广告数据集，要求使用 Lora 进行微调：

你能看到 loss 的下降，并在最终回到 3.2 左右。
你需要自己适配 inference.py 中的代码，并迁移到其他的推理框架中。例如，basic_demo 中没有读取微调模型后的 adapter 的内容，你需要参考 inference.py 的代码并进行修改，让其他 demo 能读入你的微调代码，将其部署到 basic_demo 下的 gradio_demo 中，并能够通过 webui 来进行调用。
作业提交方式：
请将相关代码和微调成功后的效果截图上传至 GitHub 或其他公开链接，然后将链接复制到下方的作业提交框提交即可。

# 作业步骤

1. 官方实例代码训练到 3.2 *左右*

![](./imgs/3.png)

2. 让 gradio_demo 加载 lora 微调模型在 webui 下调用

原模型调用的效果：
![](./imgs/1.png)

微调模型调用的效果：
![](./imgs/2.png)

3. 修改的代码 （判断模型是否有 'adapter_config.json' ，如果有就用 `AutoPeftModelForCausalLM.from_pretrained` 加载模型，否则就用默认的 `AutoModelForCausalLM.from_pretrained` 加载模型

```python
def load_model_and_tokenizer(
        model_dir: Union[str, Path], trust_remote_code: bool = True
) -> tuple[ModelType, TokenizerType]:
    model_dir = _resolve_path(model_dir)
    if (model_dir / 'adapter_config.json').exists():
        model = AutoPeftModelForCausalLM.from_pretrained(
            model_dir, trust_remote_code=trust_remote_code, device_map='auto'
        )
        tokenizer_dir = model.peft_config['default'].base_model_name_or_path
    else:
        model = AutoModelForCausalLM.from_pretrained(
            model_dir, trust_remote_code=trust_remote_code, device_map='auto'
        )
        tokenizer_dir = model_dir
    tokenizer = AutoTokenizer.from_pretrained(
        tokenizer_dir, trust_remote_code=trust_remote_code
    )
    return model, tokenizer
```

# 单卡GPU 进行 ChatGLM3-6B模型 LORA 高效微调
本 Cookbook 将带领开发者使用 `AdvertiseGen` 对 ChatGLM3-6B 数据集进行 lora微调，使其具备专业的广告生成能力。

## 硬件需求
显存：24GB及以上（推荐使用30系或A10等sm80架构以上的NVIDIA显卡进行尝试）
内存：16GB
RAM: 2.9 /16 GB
GPU RAM: 15.5/16.0 GB

## 0. 环境检查
首先，先检查代码的运行地址，确保运行地址处于 `finetune_demo` 中。
并且，确保已经安装了 `requirements.txt`中的依赖。

> 本 demo 中，不需要使用 deepspeed, mpi4py 两个依赖，如果您安装这两个依赖遇到问题，可以不安装这两个依赖。

In [5]:
!pwd

/root/autodl-tmp/ChatGLM3/finetune_demo


In [8]:
!pip install typer

Looking in indexes: http://mirrors.aliyun.com/pypi/simple


In [10]:
!pip install nltk

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 1.5 MB/s eta 0:00:00a 0:00:01


## 1. 准备数据集
我们使用 AdvertiseGen 数据集来进行微调。从 [Google Drive](https://drive.google.com/file/d/13_vf0xRTQsyneRKdD1bZIr93vBGOczrk/view?usp=sharing) 或者 [Tsinghua Cloud](https://cloud.tsinghua.edu.cn/f/b3f119a008264b1cabd1/?dl=1) 下载处理好的 AdvertiseGen 数据集，将解压后的 AdvertiseGen 目录放到本目录的 `/data/` 下, 例如。
> /media/zr/Data/Code/ChatGLM3/finetune_demo/data

In [6]:
import json
from typing import Union
from pathlib import Path


def _resolve_path(path: Union[str, Path]) -> Path:
    return Path(path).expanduser().resolve()


def _mkdir(dir_name: Union[str, Path]):
    dir_name = _resolve_path(dir_name)
    if not dir_name.is_dir():
        dir_name.mkdir(parents=True, exist_ok=False)


def convert_adgen(data_dir: Union[str, Path], save_dir: Union[str, Path]):
    def _convert(in_file: Path, out_file: Path):
        _mkdir(out_file.parent)
        with open(in_file, encoding='utf-8') as fin:
            with open(out_file, 'wt', encoding='utf-8') as fout:
                for line in fin:
                    dct = json.loads(line)
                    sample = {'conversations': [{'role': 'user', 'content': dct['content']},
                                                {'role': 'assistant', 'content': dct['summary']}]}
                    fout.write(json.dumps(sample, ensure_ascii=False) + '\n')

    data_dir = _resolve_path(data_dir)
    save_dir = _resolve_path(save_dir)

    train_file = data_dir / 'train.json'
    if train_file.is_file():
        out_file = save_dir / train_file.relative_to(data_dir)
        _convert(train_file, out_file)

    dev_file = data_dir / 'dev.json'
    if dev_file.is_file():
        out_file = save_dir / dev_file.relative_to(data_dir)
        _convert(dev_file, out_file)


convert_adgen('data/AdvertiseGen', 'data/AdvertiseGen_fix')

## 2. 使用命令行开始微调,我们使用 lora 进行微调
接着，我们仅需要将配置好的参数以命令行的形式传参给程序，就可以使用命令行进行高效微调。

In [18]:
!CUDA_VISIBLE_DEVICES=0 NCCL_P2P_DISABLE="1" NCCL_IB_DISABLE="1" python finetune_hf.py  data/AdvertiseGen_fix /root/autodl-tmp/chatglm3-6b  configs/lora.yaml

Loading checkpoint shards: 100%|██████████████████| 7/7 [00:02<00:00,  2.61it/s]
trainable params: 1,949,696 || all params: 6,245,533,696 || trainable%: 0.0312
--> Model

--> model has 1.949696M params

Map (num_proc=16): 100%|██████| 114599/114599 [00:02<00:00, 38561.84 examples/s]
train_dataset: Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 114599
})
Map (num_proc=16): 100%|███████████| 1070/1070 [00:00<00:00, 1095.81 examples/s]
val_dataset: Dataset({
    features: ['input_ids', 'output_ids'],
    num_rows: 1070
})
Map (num_proc=16): 100%|████████████| 1070/1070 [00:01<00:00, 930.11 examples/s]
test_dataset: Dataset({
    features: ['input_ids', 'output_ids'],
    num_rows: 1070
})
--> Sanity check
           '[gMASK]': 64790 -> -100
               'sop': 64792 -> -100
          '<|user|>': 64795 -> -100
                  '': 30910 -> -100
                '\n': 13 -> -100
                  '': 30910 -> -100
                '类型': 33467 -> -100
                 '#': 3

## 3. 使用微调的数据集进行推理
在完成微调任务之后，我们可以查看到 `output` 文件夹下多了很多个`checkpoint-*`的文件夹，这些文件夹代表了训练的轮数。
我们选择最后一轮的微调权重，并使用inference进行导入。

In [19]:
!CUDA_VISIBLE_DEVICES=0 NCCL_P2P_DISABLE="1" NCCL_IB_DISABLE="1" python inference_hf.py output/checkpoint-3000/ --prompt "类型#裙*版型#显瘦*材质#网纱*风格#性感*裙型#百褶*裙下摆#压褶*裙长#连衣裙*裙衣门襟#拉链*裙衣门襟#套头*裙款式#拼接*裙款式#拉链*裙款式#木耳边*裙款式#抽褶*裙款式#不规则"

Loading checkpoint shards: 100%|██████████████████| 7/7 [00:03<00:00,  1.87it/s]
这款连衣裙采用网纱拼接的设计，增加了视觉上的层次感，彰显出时尚的气息。不规则的压褶设计，修饰了身材的曲线，增添几分性感的感觉。袖口的木耳边设计，凸显出小臂的细腻感。拉链的门襟设计，方便穿脱，方便日常穿着。裙摆的百褶设计，修饰了裙摆的线条，显瘦又优雅。


## 4. 总结
到此位置，我们就完成了使用单张 GPU Lora 来微调 ChatGLM3-6B 模型，使其能生产出更好的广告。
在本章节中，你将会学会：
+ 如何使用模型进行 Lora 微调
+ 微调数据集的准备和对齐
+ 使用微调的模型进行推理